In [4]:
import tensorflow as tf
from sequence_feature_layer import SequenceFeatures
from tensorflow import feature_column as fc
from tensorflow.keras.layers import Layer, Dense, LayerNormalization, Dropout, Embedding, Conv1D

## 0.准备工作

In [5]:
seq = fc.sequence_categorical_column_with_hash_bucket('seq', hash_bucket_size=10, dtype=tf.int64)
target = fc.sequence_categorical_column_with_hash_bucket('target', hash_bucket_size=10, dtype=tf.int64)
seq_col = fc.embedding_column(seq, dimension=8)
target_col = fc.embedding_column(target, dimension=8)
columns = [seq_col, target_col]
features={
  "seq": tf.sparse.SparseTensor(
      indices=[[0, 0], [0, 1], [1, 0], [1, 1], [2, 0]],
      values=[1100, 1101, 1102, 1101, 1103],
      dense_shape=[3, 2]),
  "target": tf.sparse.SparseTensor(
      indices=[[0, 0],[1,0],[2,0]],
      values=[1102,1103,1100],
      dense_shape=[3, 1]),

}
tf.sparse.to_dense(features['seq'])

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[1100, 1101],
       [1102, 1101],
       [1103,    0]], dtype=int32)>

In [6]:
sequence_feature_layer = SequenceFeatures(columns, name='sequence_features_input_layer')
seq_emb_dict, seq_len_dict = sequence_feature_layer(features)
seq_emb_dict.keys()

dict_keys(['seq_embedding', 'target_embedding'])

In [7]:
net = tf.concat(list(seq_emb_dict.values()), axis=1)

In [10]:
seq_mask = tf.expand_dims(tf.where(tf.sequence_mask(seq_len_dict['seq_embedding']),1.0,0.0),axis=-1)
target_mask = tf.expand_dims(tf.where(tf.sequence_mask(seq_len_dict['target_embedding']),1.0,0.0),axis=-1)
mask_ = tf.concat([seq_mask,target_mask],axis=1)
mask_

<tf.Tensor: shape=(3, 3, 1), dtype=float32, numpy=
array([[[1.],
        [1.],
        [1.]],

       [[1.],
        [1.],
        [1.]],

       [[1.],
        [0.],
        [1.]]], dtype=float32)>